# **Naive Bayes Copula on Multivariate Dataset**

In [ ]:
import numpy as np
import pandas as pd
import math 
import itertools
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
iris = load_iris()
target_names = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
data1

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [ ]:
df=pd.DataFrame(columns=data1.columns)
for i in data1.keys():
  if i!='target':
    l=[]
    for j in data1[i]:
      k=np.log1p(j)
      l.append(round(k))
    df[i]=l
  else:
    df[i]=data1[i]
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,2,2,1,0,0.0
1,2,1,1,0,0.0
2,2,1,1,0,0.0
3,2,1,1,0,0.0
4,2,2,1,0,0.0
...,...,...,...,...,...
145,2,1,2,1,2.0
146,2,1,2,1,2.0
147,2,1,2,1,2.0
148,2,1,2,1,2.0


In [ ]:
# df=df.drop(['petal length (cm)'],axis=1)
n=len(df.columns)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,2,2,1,0,0.0
1,2,1,1,0,0.0
2,2,1,1,0,0.0
3,2,1,1,0,0.0
4,2,2,1,0,0.0
...,...,...,...,...,...
145,2,1,2,1,2.0
146,2,1,2,1,2.0
147,2,1,2,1,2.0
148,2,1,2,1,2.0


Clustering

In [ ]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=3, random_state=0).fit(df)
# labels = kmeans.labels_
# print(labels)

In [ ]:
# df['target']=labels
# df

In [ ]:
X=df.iloc[:,:n-1]
Y=df['target']
l=[]
for i in Y:
  l.append(int(i))
d_f={'Target':l}
Y=pd.DataFrame(d_f)  
var=Y['Target'].values
y=pd.Series(var)

In [ ]:
class naive_bayes:
    def fit(self,X,y):
        self._data={}
        X=X.to_dict()
        self._classes=np.unique(y)
        y=y.tolist()
        self._data={}
        c=0
        p=[]
        for key,value in X.items():
          l=[]
          for i,j in value.items():
            l.append(j)
          p.append(l)
        for i in range(len(y)):
          if y[c] not in self._data.keys():
            a=[]
            for i in range(len(p)):
              a.append(p[i][c])
            self._data[y[c]]=[a]
          else:
            a=[]
            for i in range(len(p)):
              a.append(p[i][c])
            q=self._data[y[c]]
            q.append(a)
            self._data[y[c]]=q
          c+=1
        # print(self._data[2])
        
        self._combo_points={}
        for key,val in self._data.items():
          self._combo_points[key]=self._combination(val[:round(len(val)*0.3)])
        # print(self._combo_points[2][:10])
 

  
    def _combination(self,val):
      l={}
      for i in range(len(val)):
        for j in range(len(val[i])):
          # print(j)
          if j not in l.keys():
            l[j]=[val[i][j]]
          else:
            k=l[j]
            k.append(val[i][j])
            l[j]=k
      a=[]      
      for key,value in l.items():
        a.append(l[key])
      # print(len(val)**4)
      return list(itertools.product(*a))  

    
    def predict(self,X,constant):
        X=X.to_dict()
        l=[]
        for key,value in X.items():
          l.append([X[key]])

        d={}  
        for i in range(len(l)):
          for key,value in l[i][0].items():
            if key not in d.keys():
              d[key]=[value]
            else:
              a=d[key]
              a.append(value)
              d[key]=a
        # print(d)    
        y_pred= [self._predict(d[key],constant) for key,value in d.items()] #(x,y)
        return y_pred
    
    #calculate posterior for each class and returning maximum one.
    def _predict(self,data,constant):
        l={}
        p=[]
        # print(data)

        for key,val in self._combo_points.items():
          count=0 
          for j in val: 
            c=0
            tot=0
            for k in j: 
              if k>=data[c]:
                # print(data[c])
                tot+=1
              c+=1
            if tot==4:
              count+=1
          prob=(np.log1p(count)+constant)/(math.sqrt(len(val))+constant*(n-1))
          l[key]=prob
          p.append(prob)
        # print(l)
    

        for key,value in l.items():
          if value==max(p):
            return key


def accuracy(y_test,y_pred):
    return np.sum(y_pred==y_test)/len(y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
l=[0.2,0.15,0.1]
for i in l:
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=i,random_state =0)  
  model=naive_bayes()
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test,32)
  print("Accuracy at",i,":",1-i,"is",accuracy(y_test,y_pred)*100) 
  print("Kappa on our model",cohen_kappa_score(y_test,y_pred))
  print("F1 score on our model",f1_score(y_test, y_pred, average='micro'))
  print("Confuion matrix\n",confusion_matrix(y_test, y_pred))
  print("Classification report on our model\n",classification_report(y_test,y_pred,target_names=target_names))
  gnb = GaussianNB()
  y_pred = gnb.fit(X_train, y_train).predict(X_test)
  print(accuracy(y_test,y_pred)*100)
  print("Kappa-NB",cohen_kappa_score(y_test,y_pred))
  print("F1 score on NB",f1_score(y_test, y_pred, average='micro'))
  print("Confuion matrix\n",confusion_matrix(y_test, y_pred))
  print("Classification report on NB\n",classification_report(y_test,y_pred,target_names=target_names))


Accuracy at 0.2 : 0.8 is 63.33333333333333
Kappa on our model 0.367816091954023
F1 score on our model 0.6333333333333333
Confuion matrix
 [[ 6  5  0]
 [ 0 13  0]
 [ 0  6  0]]
Classification report on our model
                  precision    recall  f1-score   support

    Iris-setosa       1.00      0.55      0.71        11
Iris-versicolor       0.54      1.00      0.70        13
 Iris-virginica       0.00      0.00      0.00         6

       accuracy                           0.63        30
      macro avg       0.51      0.52      0.47        30
   weighted avg       0.60      0.63      0.56        30

60.0
Kappa-NB 0.4528875379939209
F1 score on NB 0.6
Confuion matrix
 [[11  0  0]
 [ 0  1 12]
 [ 0  0  6]]
Classification report on NB
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.08      0.14        13
 Iris-virginica       0.33      1.00      0.50         6

       accuracy       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy at 0.15 : 0.85 is 65.21739130434783
Kappa on our model 0.36986301369863006
F1 score on our model 0.6521739130434783
Confuion matrix
 [[ 4  3  0]
 [ 0 11  0]
 [ 0  5  0]]
Classification report on our model
                  precision    recall  f1-score   support

    Iris-setosa       1.00      0.57      0.73         7
Iris-versicolor       0.58      1.00      0.73        11
 Iris-virginica       0.00      0.00      0.00         5

       accuracy                           0.65        23
      macro avg       0.53      0.52      0.49        23
   weighted avg       0.58      0.65      0.57        23

52.17391304347826
Kappa-NB 0.36749999999999994
F1 score on NB 0.5217391304347826
Confuion matrix
 [[ 7  0  0]
 [ 0  0 11]
 [ 0  0  5]]
Classification report on NB
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         7
Iris-versicolor       0.00      0.00      0.00        11
 Iris-virginica       0.31      1.00      0.48 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy at 0.1 : 0.9 is 66.66666666666666
Kappa on our model 0.34782608695652173
F1 score on our model 0.6666666666666666
Confuion matrix
 [[2 1 0]
 [0 8 0]
 [0 4 0]]
Classification report on our model
                  precision    recall  f1-score   support

    Iris-setosa       1.00      0.67      0.80         3
Iris-versicolor       0.62      1.00      0.76         8
 Iris-virginica       0.00      0.00      0.00         4

       accuracy                           0.67        15
      macro avg       0.54      0.56      0.52        15
   weighted avg       0.53      0.67      0.57        15

46.666666666666664
Kappa-NB 0.2857142857142858
F1 score on NB 0.4666666666666667
Confuion matrix
 [[3 0 0]
 [0 0 8]
 [0 0 4]]
Classification report on NB
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         3
Iris-versicolor       0.00      0.00      0.00         8
 Iris-virginica       0.33      1.00      0.50         4

       ac

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)
# print(accuracy(y_test,y_pred))

In [ ]:
# from random import seed
# from random import randrange
# def nfold_split(data,num): #Here num is the number of splits to be done for the data
#   chunk = list()
#   dataset = data.values.tolist()
#   fold_size = int(len(data)/num)
#   for i in range(num):
#     fold = list()
#     while len(fold) < fold_size:
#       index = randrange(len(dataset))
#       fold.append(dataset.pop(index))
#     chunk.append(fold)
#   return chunk

In [ ]:
# seed(1)
# chunks = nfold_split(df, 10)
# # print(chunks)
# y_pred = 0
# our_model_acc= []
# gauss_acc=[]
# model=naive_bayes()
# MSE1sk = []
# for i in range(10):
#   df_train = pd.DataFrame()
#   for j in range(10):
#     if (j==i):
#       df_test = pd.DataFrame(chunks[j])
#     else:
#       #train.append(chunks[j])
#       data1 = pd.DataFrame()
#       data1 = pd.DataFrame(chunks[j])
#       df_train = df_train.append(data1, ignore_index=True)
#   # print(df_train)
#   pp=pd.DataFrame(df_train.iloc[:,4:])
#   Y=pp.values
#   Y=Y.reshape(-1)
#   y_train=pd.Series(Y)
#   model.fit(df_train.iloc[:,0:4], y_train)
#   y_pred = model.predict(df_test.iloc[:,0:4],32)
#   pp=pd.DataFrame(df_test.iloc[:,4:])
#   Y=pp.values
#   Y=Y.reshape(-1)
#   y_test=pd.Series(Y)
#   print(y_test)
#   score = accuracy(y_test, y_pred)
#   our_model_acc.append(score)
#   gnb = GaussianNB()
#   y_pred = gnb.fit(df_train.iloc[:,0:4], y_train).predict(df_test.iloc[:,0:4])
#   gauss_acc.append(accuracy(y_test,y_pred))
# our_model_acc,gauss_acc

In [ ]:
# data1